In [1]:
import os
import pydicom
import pandas as pd

# Folder paths
input_folder = r"C:\Users\A!\Desktop\002Y"
output_file = r"C:\Users\A!\Desktop\New folder (6)\dicom_metadata.xlsx"

# Initialize a list to store metadata
metadata_list = []

# Loop through each patient folder
for root, dirs, files in os.walk(input_folder):
    # Look for DICOM files in the folder
    dicom_files = [f for f in files if f.lower().endswith('.dcm')]
    
    if dicom_files:
        # Select the first DICOM file from the folder
        dicom_file_path = os.path.join(root, dicom_files[0])
        
        # Read the DICOM file
        dicom_data = pydicom.dcmread(dicom_file_path)
        
        # Initialize a dictionary for storing this patient's metadata
        patient_metadata = {}
        
        # Iterate through all available DICOM data elements
        for elem in dicom_data:
            # Get the tag name (keyword) if available, otherwise use the tag number
            tag_name = pydicom.datadict.keyword_for_tag(elem.tag)
            if not tag_name:
                tag_name = f"Unknown_{elem.tag}"
            
            # Add the tag name and value to the metadata dictionary
            tag_value = elem.value if elem.VR != 'SQ' else 'Sequence'
            patient_metadata[tag_name] = tag_value
        
        # Add the patient's metadata to the list
        metadata_list.append(patient_metadata)

# Create a DataFrame from the metadata list
df = pd.DataFrame(metadata_list)

# Save the DataFrame to an Excel file
df.to_excel(output_file, index=False)

print(f"Metadata saved to {output_file}")


OSError: Cannot save file into a non-existent directory: 'C:\Users\A!\Desktop\New folder (6)'

edit: yek radif esm va tartib dodrost sotomha

In [2]:
import os
import pydicom
import pandas as pd

# Folder paths
input_folder = r"C:\Users\A!\Desktop\002Y"
output_file = r"C:\Users\A!\Desktop\New folder (6)\dicom_metadata.xlsx"

# Initialize a dictionary to store metadata (using PatientID to avoid duplicates)
patient_metadata_dict = {}

# Loop through each patient folder
for root, dirs, files in os.walk(input_folder):
    # Look for DICOM files in the folder
    dicom_files = [f for f in files if f.lower().endswith('.dcm')]
    
    if dicom_files:
        # Select the first DICOM file from the folder
        dicom_file_path = os.path.join(root, dicom_files[0])
        
        # Read the DICOM file
        dicom_data = pydicom.dcmread(dicom_file_path)
        
        # Extract patient ID to avoid duplicates
        patient_id = getattr(dicom_data, 'PatientID', 'Unknown')
        
        # If the patient ID is already processed, skip adding another row
        if patient_id in patient_metadata_dict:
            continue
        
        # Initialize a dictionary for storing this patient's metadata
        patient_metadata = {}
        
        # Extract specific fields in the required order
        patient_metadata['PatientName'] = getattr(dicom_data, 'PatientName', 'Unknown')
        patient_metadata['PatientID'] = patient_id
        patient_metadata['PatientAge'] = getattr(dicom_data, 'PatientAge', 'Unknown')
        patient_metadata['PatientSex'] = getattr(dicom_data, 'PatientSex', 'Unknown')
        
        # Iterate through all available DICOM data elements and add the rest of the metadata
        for elem in dicom_data:
            tag_name = pydicom.datadict.keyword_for_tag(elem.tag)
            if not tag_name:
                tag_name = f"Unknown_{elem.tag}"
            
            # Add the tag name and value to the metadata dictionary (excluding already captured fields)
            if tag_name not in patient_metadata:
                tag_value = elem.value if elem.VR != 'SQ' else 'Sequence'
                patient_metadata[tag_name] = tag_value
        
        # Add this patient's metadata to the dictionary
        patient_metadata_dict[patient_id] = patient_metadata

# Create a DataFrame from the patient metadata dictionary
df = pd.DataFrame(patient_metadata_dict.values())

# Reorder columns: PatientName, PatientID, PatientAge, PatientSex
desired_columns = ['PatientName', 'PatientID', 'PatientAge', 'PatientSex']
other_columns = [col for col in df.columns if col not in desired_columns]
df = df[desired_columns + other_columns]

# Save the DataFrame to an Excel file
df.to_excel(output_file, index=False)

print(f"Metadata saved to {output_file}")


Metadata saved to C:\Users\A!\Desktop\New folder (6)\dicom_metadata.xlsx
